In [51]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import opensmile
import IPython
import os
import librosa
import scipy.io.wavfile as wav
import statistics


In [17]:
mydir=("C:/Users/admul/Desktop/School/Thesis/DAICWOZ Data/Patient Audio")

In [21]:
filelist=[]
for path, subdirs, files in os.walk(mydir):
    for file in files:
        if (file.endswith('.wav') or file.endswith('.WAV'))  :
            filelist.append(os.path.join(path, file))
number_of_files=len(filelist)


In [126]:
duration_mean_all_list = []
duraiton_sd_all_list = []
overlap_mean_list = []
overlap_sd_list = []
duration_post_mean_list = []
duration_post_sd_list = []
patient_num_list = []

for i in range(len(filelist)):

    subpath = filelist[i][:-14]
    patient_number = filelist[i][70:73]
    

    dataTranscript= pd.read_csv(subpath + "/" + patient_number + "_TRANSCRIPT.CSV", sep='\t', header=0)
    dataDuration =dataTranscript[["speaker", "start_time", "stop_time"]]
    
    duration = []

    
    first_ellie_detect = 0
    lastSpeaker = dataTranscript['speaker'][0]

    duration_speaker = []
        
    for row in range(len(dataDuration)):      
                       
        if dataTranscript['speaker'][row] == "Participant" and lastSpeaker == "Ellie":
            stop_time_last_speaker = dataTranscript['stop_time'][row -1]
            start_time = dataTranscript['start_time'][row]
            
            pause_time = start_time - stop_time_last_speaker
            
            #print("row is " +str(row+1) + " pause time was " + str(pause_time) +" with start time: " + str (start_time) +"and end time " + str(stop_time_last_speaker))
            
            duration_speaker.append(pause_time)
            
                
        lastSpeaker = dataTranscript['speaker'][row]
    #print(i)
    if duration_speaker == []:
        patient_num_list.append(patient_number)
        duration_mean_all_list.append("Null")
        duraiton_sd_all_list.append("Null")
        overlap_mean_list.append("Null")
        overlap_sd_list.append("Null")
        duration_post_mean_list.append("Null")
        duration_post_sd_list.append("Null")
    if duration_speaker != []:
    
        patient_num_list.append(patient_number)
        overlap = [item for item in duration_speaker if item < 0]
        response = [item for item in duration_speaker if item >= 0]

        duration_mean_all_p = statistics.mean(duration_speaker)
        duration_mean_all_list.append(duration_mean_all_p)

        duraiton_sd_all_p = statistics.pstdev(duration_speaker)
        duraiton_sd_all_list.append(duraiton_sd_all_p)

        if overlap != []:
            overlap_mean_p = statistics.mean( abs(num) for num in overlap)
            overlap_mean_list.append(overlap_mean_p)

            overlap_sd_p = statistics.pstdev(overlap)
            overlap_sd_list.append(overlap_sd_p)
        if overlap == []:
            overlap_mean_list.append("Null")
            overlap_sd_list.append("Null")

        duration_post_all_p = statistics.mean(response)
        duration_post_mean_list.append(duration_post_all_p)

        duration_post_sd_p = statistics.pstdev(response)
        duration_post_sd_list.append(duration_post_sd_p)
        


    
    

In [127]:

dataDuration = pd.DataFrame(np.column_stack([patient_num_list, duration_mean_all_list, duraiton_sd_all_list,
                                            overlap_mean_list, overlap_sd_list, duration_post_mean_list,
                                            duration_post_sd_list]), 
                                   columns=['patient_id', 'response_dur_mean', 'response_dur_sd', 'overlap_mean', 'overlap_sd',
                                           'response_no_overlap_mean', 'response_no_overlap_sd'])

In [135]:
dataDuration.tail(50)

,patient_id,response_dur_mean,response_dur_sd,overlap_mean,overlap_sd,response_no_overlap_mean,response_no_overlap_sd
139,442,0.9396909090909122,1.2840000265183769,0.6910000000000007,1.0567757567242047,1.21725531914894,1.0997841851750214
140,443,0.9226166666666666,0.7559221100013459,0.8952499999999972,0.8993142873878895,1.0524642857142854,0.5491436503543398
141,444,0.6183090909090889,1.2844767576045917,0.7612307692307638,0.8150484793364414,1.0453095238095196,1.087909211200094
142,445,1.0540599999999976,1.5266694915403252,2.2837499999999977,2.8526598443382625,1.344304347826084,0.8790594002179426
143,446,0.7583043478260812,0.8612620635758879,0.901250000000001,1.4378748858993178,0.9163571428571367,0.572919001232093
144,447,0.9836730769230779,0.9371702439782327,0.3709999999999927,0.21674562663792762,1.0666122448979598,0.8999723310106552
145,448,0.6623513513513509,1.7565626017451037,1.2731250000000074,2.211846538387103,1.196275862068967,1.1263320664440437
146,449,0.005302325581391254,1.7906007765991188,3.3209999999999895,2.9196941049820158,0.5447027027026963,0.5085788819660856
147,450,0.47835937499999726,0.9060410167175702,0.6393125000000057,0.6318940495397518,0.8509166666666649,0.6373810945754659
148,451,Null,Null,Null,Null,Null,Null


In [131]:
dataDuration.to_csv('C:/Users/admul/Desktop/School/Thesis/DAICWOZ Data/Altered Audio/DurationData.csv', index=False)